In [1]:
import gc
import os

import keras
import keras_models
import numpy as np
import pandas as pd
import utils
from gensim.models import KeyedVectors
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from keras.optimizers import SGD, Adadelta, Adam, Nadam, RMSprop
from keras.preprocessing import sequence, text
from nltk.corpus import stopwords
from tqdm import tqdm

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

Using TensorFlow backend.


In [2]:
n_folds = 5
n_bags = 1
split_size = 0.1
max_features = 300000
nb_words = max_features
sequence_length = 196
embedding_dim = 300
bidirectional = True
run_prefix = '256_FastText300k_'
embedding_filename = 'FastText_300dim_embeddingBasic300k'

run_prefix = 'FastText_'
src = '/home/w/Projects/Toxic/data/'
model_name = 'MLPbasic'
optimizer = 'Nadam'
data_type = 'SpacyClean'
kfold_run = 0
batch_size = 256
importance = 0
stratify = 0
save_models = 0
load_models = 0
save_oof = 0
prepare_submission = 1


if bidirectional and 'LSTM' in model_name or bidirectional and 'GRU' in model_name:
    run_prefix = 'Bidirectional{}'.format(run_prefix)
if kfold_run:
    general_run_name = '{}{}fold_BS{}_{}'.format(
        run_prefix, n_folds, batch_size, optimizer)
else:
    general_run_name = '{}{}bag_BS{}_{}'.format(
        run_prefix, n_bags, batch_size, optimizer)


if len(data_type) > 0:
    general_run_name += '_{}'.format(data_type)
if importance:
    general_run_name += '_ImportanceTrain'
if stratify and kfold_run:
    general_run_name += '_Stratified'

run_name = '{}{}'.format(model_name, general_run_name)
print('Run name: {}'.format(run_name))


model_callbacks = [EarlyStopping(monitor='val_loss', patience=18, verbose=1),
                   ReduceLROnPlateau(monitor='val_loss', factor=0.5, verbose=1,
                                     patience=8, min_lr=1e-5)]

if optimizer == 'Adam':
    optimizer = Adam(lr=1e-4, decay=1e-3)
    # optimizer = 'adam'
if optimizer == 'Nadam':
    optimizer = Nadam(lr=1e-4, schedule_decay=1e-3)
    # optimizer = 'nadam'
if optimizer == 'SGD':
    optimizer = SGD(lr=1e-3, momentum=0.9,
                    decay=1e-4, nesterov=True)

Run name: MLPbasicFastText_1bag_BS256_Nadam_SpacyClean


In [3]:
src_features = '/home/w/Projects/Toxic/data/features/'
run_kfold = True
split_for_validation = True

if run_kfold:
    prefix = 'KFold'
else:
    prefix = 'Bag'


train = pd.read_pickle("../data/train_basic_clean.pkl")
test = pd.read_pickle("../data/test_basic_clean.pkl")
target_columns = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']


data_tokenized = pd.read_pickle(src_features + 'data_TokenizedSentences196.pkl')
data_badwords300 = pd.read_pickle(src_features + 'data_Binary300Badwords.pkl')
data_badwordsCount = pd.read_pickle(src_features + 'data_BadwordsCount.pkl')
data_textStatistics = pd.read_pickle(src_features + 'data_TextStatistics.pkl')
data_transformations = pd.read_pickle(src_features + 'data_TransformationsFeats20dim_SVDLSA.pkl')

X = pd.concat([data_tokenized, data_badwords300, data_textStatistics, data_transformations], axis=1)
X['badwordsCount'] = data_badwordsCount

X_train = X.iloc[:train.shape[0], :]
X_test = X.iloc[train.shape[0]:, :]
y_train = train[target_columns]

features = np.setdiff1d(X_train.columns, target_columns)



del X, test
del data_tokenized, data_badwords300, data_badwordsCount
del data_textStatistics, data_transformations
gc.collect()

30

In [4]:
model_parameters = {
    'lstm_units': 256,
    'bidirectional': bidirectional,
    'nb_words': nb_words,
    'embedding_dim': embedding_dim,
    #'embedding_matrix': embedding_matrix,
    'sequence_length': sequence_length,
    'optimizer': optimizer,
    'num_columns': X_train.shape[1],
}

pipeline_parameters = {
    'model_name': getattr(keras_models, model_name),
    'predict_test': True,
    'number_epochs': 1000,
    'batch_size': batch_size,
    'seed': 1337,
    'shuffle': True,
    'verbose': True,
    'run_save_name': run_name,
    'load_keras_model': load_models,
    'save_model': save_models,
    'save_history': True,
    'save_statistics': True,
    'output_statistics': True,
    'src_dir': os.getcwd(),
}

if kfold_run:
    oof_train, oof_test = utils.run_parametrized_kfold(X_train[features], y_train, 
                                                       X_test[features],
                                                       pipeline_parameters,
                                                       model_parameters,
                                                       model_callbacks=model_callbacks,
                                                       n_folds=n_folds,
                                                       importance_training=importance,
                                                       save_oof=save_oof)
    print(oof_train.shape, oof_test.shape)
else:
    oof_valid, oof_test = utils.run_parametrized_bagging(X_train[features], y_train,
                                                         X_test=X_test[features],
                                                         pipeline_parameters=pipeline_parameters,
                                                         model_parameters=model_parameters,
                                                         model_callbacks=model_callbacks,
                                                         n_bags=n_bags,
                                                         split_size=split_size,
                                                         importance_training=importance)
    print(oof_valid.shape, oof_test.shape)


if prepare_submission:
    submission = utils.output_submission(
        oof_test.mean(axis=0), run_name, save=True)

Running parametrized bagging
Running: MLPbasicFastText_1bag_BS256_Nadam_SpacyClean
Training on bag: 1 

Saving CSV logs for model from current bag/fold: MLPbasicFastText_1bag_BS256_Nadam_SpacyClean, bag number 1 

Splitting data - validation split size: 0.1, split seed: 1337
Train on 86265 samples, validate on 9586 samples
Epoch 1/1000
86265/86265 [==============================] - 3s 40us/step - loss: 0.2176 - acc: 0.9273 - val_loss: 0.0942 - val_acc: 0.9733
Epoch 2/1000
86265/86265 [==============================] - 3s 29us/step - loss: 0.0833 - acc: 0.9759 - val_loss: 0.0699 - val_acc: 0.9787
Epoch 3/1000
86265/86265 [==============================] - 3s 29us/step - loss: 0.0758 - acc: 0.9770 - val_loss: 0.0674 - val_acc: 0.9795
Epoch 4/1000
86265/86265 [==============================] - 3s 30us/step - loss: 0.0726 - acc: 0.9775 - val_loss: 0.0660 - val_acc: 0.9797
Epoch 5/1000
86265/86265 [==============================] - 3s 30us/step - loss: 0.0711 - acc: 0.9780 - val_loss: 0.066